In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [206]:
def Thomas_Algorithm(a, b, c, d):

    ne = len(d) # number of equations
    ac, bc, cc, dc = map(np.array, (a, b, c, d)) # copy arrays
    for it in range(1, ne):   
        ac[it] = a[it] - b[it]*(c[it-1]/ac[it-1]) 
        dc[it] = d[it] - b[it]*(dc[it-1]/ac[it-1])

    xc = np.zeros(ne)
    xc[-1] = dc[-1]/ac[-1]
    for il in range(ne-2, -1, -1):
        xc[il] = (dc[il]/ac[il]) - (c[il]/ac[il])*xc[il+1]

    return xc 

## Summary

1. Name : TRUE ELS 15365회 (SK하이닉스, NAVER)
2. Payoff : [0.051, 0.102, 0.153, 0.204, 0.255, 0.306] and 이론가 : 8,863.69 / based on 10000원
3. parameter : <br>
early redemption : [0.85, 0.85, 0.80, 0.80, 0.75, 0.75] <br>
ending criteria : principle x min(maturity price/initial price) <br>
volatility : SK하이닉스(38.88%), NAVER(39.62%), 상관계수(0.5287) 180의 역사적 상관계수임 <br>


In [813]:
'''From Market Condition'''
r = 0.0282 #발행일 8월 31일의 90일 CD금리

sk_q = 0.0162 #8월 31일의 sk하이닉스 배당수익률
na_q = 0.0022 #8월 31일의 네이버 배당수익률

rho = 0.5287 #투자 설명서의 상관계수
sk_sigma = 0.3888 #투자설명서의 SK하이닉스 변동성
na_sigma = 0.3962 #투자설명서의 NAVER 변동성

sk_0 = 1 #계산의 편의를 위해, 현재시점의 SK하이닉스의 가격을 1로 잡음
na_0 = 1 #계산의 편의를 위해, 현재시점의 NAVER의 가격을 1로 잡음

sk_mu = r - sk_q - rho * sk_sigma * na_sigma
na_mu = r - na_q - rho * na_sigma * sk_sigma

'''Given tags'''
B = 0.5
ctag = np.array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])
ktag = np.array([0.85, 0.85, 0.80, 0.80, 0.75, 0.75])
ttag = np.array([0.5, 1, 1.5, 2, 2.5, 3])

'''For Mesh'''
T = 3 #만기 3년

sk_max = sk_0 * 5
na_max = na_0 * 5
# sk_max = sk_0 * ( 1 + T * 3 * sk_sigma) #SK하이닉스의 max 바운더리
# na_max = na_0 * ( 1 + T * 3 * na_sigma) #NAVER의 max 바운더리

M = 60 #가격 노드 나누기 (y축)
N = 300 #시간 노드 나누기 (x축)

'''h와, k생성 > 이거 나중에 값 대체하면됨, 교과서 따라가려고 일케 해둠'''
dt = T/N #시간노드 간격 
k = dt #교과서 Notation 따라가기

sk_dx = sk_max/(M*sk_0) #SK하이닉스의 가격노드 간격
na_dx = na_max/(M*na_0) #NAVER의 가격노드 간격
h = sk_dx #교과서 Notation 따라가기, 계산의 편의를 위해 sk_max와, na_max를 같게해서 dx가 같게함
xh = np.arange(M+1)
x = xh * sk_dx 

a = k/(h**2) #alpha

x0tag = np.where(x==sk_0)[0][0]
Btag = np.where(x==B)[0][0]
Rtag = 2 * N/len(ktag) * ttag

In [800]:
'''Mesh 좌표 간격'''
print(f'Mesh point interval : {(dt, sk_dx, na_dx)}')

Mesh point interval : (0.01, 0.08333333333333333, 0.08333333333333333)


In [823]:
x = np.full((M+1, M+1), (np.arange(M+1))) / M

u = np.zeros((N+1, M+1, M+1)) #시간, SK하이닉스, NAVER의 노드
w = np.zeros((N+1, M+1, M+1))

u[0, :] = np.where(x>B, 1 + ctag[-1], x).reshape(M+1, M+1)
w[0, :] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(M+1, M+1)

u[:, 0, :] = 2*u[:, 1, :] - u[:, 2, :]
u[:, M, :] = 2*u[:, M-1, :] - u[:, M-2, :]
u[:, :, 0] = 2*u[:, :, 1] - u[:, :, 2]
u[:, :, M] = 2*u[:, :, M-1] - u[:, :, M-2]

w[:, 0, :] = 2*w[:, 1, :] - w[:, 2, :]
w[:, M, :] = 2*w[:, M-1, :] - w[:, M-2, :]
w[:, :, 0] = 2*w[:, :, 1] - w[:, :, 2]
w[:, :, M] = 2*w[:, :, M-1] - w[:, :, M-2]

corr_term = (rho*sk_sigma*na_sigma*xh**2)/(8*h**2)

a_x = 1 + k*(r/2 + (r-sk_q) * (xh)/h + (sk_sigma*xh)**2/h**2)
b_x = -1/2 * k * (sk_sigma*xh)**2/h**2
c_x = -k*((r-sk_q) * xh/h + 0.5*(sk_sigma*xh)**2/h**2)

a_y = 1 + k*(r/2 + (r-na_q) * (xh)/h + (na_sigma*xh)**2/h**2)
b_y = -1/2 * k *( na_sigma*xh)**2/h**2
c_y = -k*((r-na_q) * xh/h + 0.5*(na_sigma*xh)**2/h**2)

corr_term = corr_term

# a_x = a_x[1:M]
# b_x = b_x[1:M]
# c_x = c_x[1:M]
# a_y = a_y[1:M]
# a_y = a_y[1:M]
# a_y = a_y[1:M]

u_star = np.zeros((N+1, M+1, M+1))
w_star = np.zeros((N+1, M+1, M+1))

# t_idx, x_idx, y_idx = 0, 1, 1
# u[t_idx, x_idx, y_idx] + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1])

for t_idx in range(0,N): 

    '''for early redemption'''
    for j in range(len(ktag)-1):
        if t_idx == Rtag[j]:
            dummy = int(x0tag * ctag[len(ctag)-2-j])
            
            u[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]
            w[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]

    for y_idx in range(1, M-1):
        d_u = u[t_idx, 1:M, y_idx] + corr_term[1:M]*(u[t_idx, 2:, y_idx+1]+u[t_idx, 0:M-1, y_idx-1]-u[t_idx, 0:M-1, y_idx+1]-u[t_idx, 2:, y_idx-1])
        u_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_u)

        d_w = w[t_idx, 1:M, y_idx] + corr_term[1:M]*(w[t_idx, 2:, y_idx+1]+w[t_idx, 0:M-1, y_idx-1]-w[t_idx, 0:M-1, y_idx+1]-w[t_idx, 2:, y_idx-1])
        w_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_w)



    for x_idx in range(1, M-1):
        d_u = u_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(u_star[t_idx, x_idx+1, 2:]+u_star[t_idx, x_idx-1, 0:M-1]-u_star[t_idx, x_idx-1, 2:]-u_star[t_idx, x_idx+1, 0:M-1])
        u[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_u)
        
        d_w = w_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(w_star[t_idx, x_idx+1, 2:]+w_star[t_idx, x_idx-1, 0:M-1]-w_star[t_idx, x_idx-1, 2:]-w_star[t_idx, x_idx+1, 0:M-1])
        w[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_w)

    u[t_idx+1, 1:Btag, 1:Btag] = w[t_idx+1, 1:Btag, 1:Btag]
    
    u[t_idx+1, 0, :] = 2*u[t_idx+1, 1, :] - u[t_idx+1, 2, :]
    u[t_idx+1, M, :] = 2*u[t_idx+1, M-1, :] - u[t_idx+1, M-2, :]
    u[t_idx+1, :, 0] = 2*u[t_idx+1, :, 1] - u[t_idx+1, :, 2]
    u[t_idx+1, :, M] = 2*u[t_idx+1, :, M-1] - u[t_idx+1, :, M-2]

    w[t_idx+1, 0, :] = 2*w[t_idx+1, 1, :] - w[t_idx+1, 2, :]
    w[t_idx+1, M, :] = 2*w[t_idx+1, M-1, :] - w[t_idx+1, M-2, :]
    w[t_idx+1, :, 0] = 2*w[t_idx+1, :, 1] - w[t_idx+1, :, 2]
    w[t_idx+1, :, M] = 2*w[t_idx+1, :, M-1] - w[t_idx+1, :, M-2]

u[N,x0tag, x0tag], w[N,x0tag, x0tag]

(0.0, 0.0)

In [830]:
for x in range(N):
    print(u[x, x0tag, x0tag])

0.18619861911515206
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.285021437287709
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.1983701184081554
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.1468420552351648
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0961144361397128
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

In [815]:
corr_term = (rho*sk_sigma*na_sigma*xh**2)/(8*h**2)

a_x = 1 + k*(r/2 + (r-sk_q) * (xh)/h + (sk_sigma*xh)**2/h**2)
b_x = -1/2 * k * (sk_sigma*xh)**2/h**2
c_x = -k*((r-sk_q) * xh + 0.5*(sk_sigma*xh)**2/h**2)

a_y = 1 + k*(r/2 + (r-na_q) * (xh)/h + (na_sigma*xh)**2/h**2)
b_y = -1/2 * k *( na_sigma*xh)**2/h**2
c_y = -k*((r-na_q) * xh + 0.5*(na_sigma*xh)**2/h**2)

corr_term = corr_term

# a_x = a_x[1:M]
# b_x = b_x[1:M]
# c_x = c_x[1:M]
# a_y = a_y[1:M]
# a_y = a_y[1:M]
# a_y = a_y[1:M]

u_star = np.zeros((N+1, M+1, M+1))
w_star = np.zeros((N+1, M+1, M+1))

# t_idx, x_idx, y_idx = 0, 1, 1
# u[t_idx, x_idx, y_idx] + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1])

for t_idx in range(0,N): 

    '''for early redemption'''
    for j in range(len(ktag)-1):
        if t_idx == Rtag[j]:
            dummy = int(x0tag * ctag[len(ctag)-2-j])
            
            u[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]
            w[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]

    for y_idx in range(1, M-1):
        d_u = u[t_idx, 1:M, y_idx] + corr_term[1:M]*(u[t_idx, 2:, y_idx+1]+u[t_idx, 0:M-1, y_idx-1]-u[t_idx, 0:M-1, y_idx+1]-u[t_idx, 2:, y_idx-1])
        u_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_u)

        d_w = w[t_idx, 1:M, y_idx] + corr_term[1:M]*(w[t_idx, 2:, y_idx+1]+w[t_idx, 0:M-1, y_idx-1]-w[t_idx, 0:M-1, y_idx+1]-w[t_idx, 2:, y_idx-1])
        w_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_w)



    for x_idx in range(1, M-1):
        d_u = u_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(u_star[t_idx, x_idx+1, 2:]+u_star[t_idx, x_idx-1, 0:M-1]-u_star[t_idx, x_idx-1, 2:]-u_star[t_idx, x_idx+1, 0:M-1])
        u[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_u)
        
        d_w = w_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(w_star[t_idx, x_idx+1, 2:]+w_star[t_idx, x_idx-1, 0:M-1]-w_star[t_idx, x_idx-1, 2:]-w_star[t_idx, x_idx+1, 0:M-1])
        w[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_w)

    u[t_idx+1, 1:Btag, 1:Btag] = w[t_idx+1, 1:Btag, 1:Btag]
    
    u[t_idx+1, 0, :] = 2*u[t_idx+1, 1, :] - u[t_idx+1, 2, :]
    u[t_idx+1, M, :] = 2*u[t_idx+1, M-1, :] - u[t_idx+1, M-2, :]
    u[t_idx+1, :, 0] = 2*u[t_idx+1, :, 1] - u[t_idx+1, :, 2]
    u[t_idx+1, :, M] = 2*u[t_idx+1, :, M-1] - u[t_idx+1, :, M-2]

    w[t_idx+1, 0, :] = 2*w[t_idx+1, 1, :] - w[t_idx+1, 2, :]
    w[t_idx+1, M, :] = 2*w[t_idx+1, M-1, :] - w[t_idx+1, M-2, :]
    w[t_idx+1, :, 0] = 2*w[t_idx+1, :, 1] - w[t_idx+1, :, 2]
    w[t_idx+1, :, M] = 2*w[t_idx+1, :, M-1] - w[t_idx+1, :, M-2]



In [779]:
Btag

6

In [784]:
ctag

array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])

In [786]:
1 + ctag[len(ctag)-2-0]

1.255

In [816]:
u_star = np.zeros((N+1, M+1, M+1))
w_star = np.zeros((N+1, M+1, M+1))

# t_idx, x_idx, y_idx = 0, 1, 1
# u[t_idx, x_idx, y_idx] + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1])

for t_idx in range(0,N): 

    '''for early redemption'''
    for j in range(len(ktag)-1):
        if t_idx == Rtag[j]:
            dummy = int(x0tag * ctag[len(ctag)-2-j])
            
            u[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]
            w[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]

    for y_idx in range(1, M-1):
        d_u = u[t_idx, 1:M, y_idx] + corr_term[1:M]*(u[t_idx, 2:, y_idx+1]+u[t_idx, 0:M-1, y_idx-1]-u[t_idx, 0:M-1, y_idx+1]-u[t_idx, 2:, y_idx-1])
        u_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_u)

        d_w = w[t_idx, 1:M, y_idx] + corr_term[1:M]*(w[t_idx, 2:, y_idx+1]+w[t_idx, 0:M-1, y_idx-1]-w[t_idx, 0:M-1, y_idx+1]-w[t_idx, 2:, y_idx-1])
        w_star[t_idx, 1:M, y_idx] = Thomas_Algorithm(a_x, b_x, c_x, d_w)



    for x_idx in range(1, M-1):
        d_u = u_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(u_star[t_idx, x_idx+1, 2:]+u_star[t_idx, x_idx-1, 0:M-1]-u_star[t_idx, x_idx-1, 2:]-u_star[t_idx, x_idx+1, 0:M-1])
        u[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_u)
        
        d_w = w_star[t_idx, x_idx, 1:M] + corr_term[1:M]*(w_star[t_idx, x_idx+1, 2:]+w_star[t_idx, x_idx-1, 0:M-1]-w_star[t_idx, x_idx-1, 2:]-w_star[t_idx, x_idx+1, 0:M-1])
        w[t_idx, x_idx, 1:M] = Thomas_Algorithm(a_x, b_x, c_x, d_w)

    u[t_idx+1, 1:Btag, 1:Btag] = w[t_idx+1, 1:Btag, 1:Btag]
    
    u[t_idx+1, 0, :] = 2*u[t_idx+1, 1, :] - u[t_idx+1, 2, :]
    u[t_idx+1, M, :] = 2*u[t_idx+1, M-1, :] - u[t_idx+1, M-2, :]
    u[t_idx+1, :, 0] = 2*u[t_idx+1, :, 1] - u[t_idx+1, :, 2]
    u[t_idx+1, :, M] = 2*u[t_idx+1, :, M-1] - u[t_idx+1, :, M-2]

    w[t_idx+1, 0, :] = 2*w[t_idx+1, 1, :] - w[t_idx+1, 2, :]
    w[t_idx+1, M, :] = 2*w[t_idx+1, M-1, :] - w[t_idx+1, M-2, :]
    w[t_idx+1, :, 0] = 2*w[t_idx+1, :, 1] - w[t_idx+1, :, 2]
    w[t_idx+1, :, M] = 2*w[t_idx+1, :, M-1] - w[t_idx+1, :, M-2]



12

In [822]:
u[N,x0tag, x0tag], w[N,x0tag, x0tag]

(0.0, 0.0)

In [518]:
t_idx, x_idx, y_idx = 0, 1, 1
ppp = u[t_idx, x_idx, y_idx] + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1])
ppp.shape

(101,)

In [520]:
u_star[0, 1, 1] = Thomas_Algorithm(a_x, b_x, c_x, ppp)

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [506]:
u_star = np.zeros((N, M+1, M+1))

for t_idx in range (0, N):
    for y_idx in range (1, M-1):
        for x_idx in range (1, M-1):
            d_xy = u[t_idx, x_idx, y_idx] + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1])
            u_star = Thomas_Algorithm(a_x, b_x, c_x, d_xy)
        
        


KeyboardInterrupt: 

In [500]:
t_idx, y_idx = 0, 1

d_x = u[t_idx, :, y_idx][1:M]/k + corr_term*(u[t_idx, :, y_idx+1][2:] + u[t_idx, :, y_idx-1][0:M-1] - u[t_idx, :, y_idx+1][0:M-1] - u[t_idx, :, y_idx-1][2:])
#d_x
#u_star[t_idx,:,y_idx] =
Thomas_Algorithm(a_x, b_x, c_x, d_x)

(99,)

In [456]:
for t_idx in range(0, N):
    
    for x_idx in range(1, M):
        
        for y_idx in range(1, M):

            d_x = (u[t_idx, x_idx, y_idx]/k + corr_term*(u[t_idx, x_idx+1, y_idx+1] + u[t_idx, x_idx-1, y_idx-1] - u[t_idx, x_idx-1, y_idx+1] - u[t_idx, x_idx+1, y_idx-1]))
            u_start = Thomas_Algorithm(a_x, b_x, c_x, d_x)

            d_y = 

            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
5

In [ ]:
a = 1 + k

In [395]:
'''OSFDM'''

'''xh가 xi/h 가 맞는지? h로 안나눠도 되는지'''

a_1 = 1 + k * (r/2 + (sk_mu * xh) + ((sk_sigma*xh)**2))
b_1 = (-1) * k*((sk_sigma*xh)**2)/(2)
c_1 = -k * ((sk_mu * xh) + (sk_sigma*xh)**2/(2))

a_2 = 1 + k * (r/2 + (na_mu * xh)/h + ((na_sigma*xh)**2))
b_2 = (-1) * k*((na_sigma*xh)**2)/(2)
c_2 = -k * ((na_mu * xh)/h + (na_sigma*xh)**2/(2))


In [419]:
x = np.full((M+1, M+1), (np.arange(M+1))) / M

u = np.zeros((N+1, M+1, M+1)) #시간, SK하이닉스, NAVER의 노드
w = np.zeros((N+1, M+1, M+1))

u[0, :] = np.where(x>B, 1 + ctag[-1], x).reshape(M+1, M+1)
w[0, :] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(M+1, M+1)

u[:, 0, :] = 2*u[:, 1, :] - u[:, 2, :]
u[:, M, :] = 2*u[:, M-1, :] - u[:, M-2, :]
u[:, :, 0] = 2*u[:, :, 1] - u[:, :, 2]
u[:, :, M] = 2*u[:, :, M-1] - u[:, :, M-2]

w[:, 0, :] = 2*w[:, 1, :] - w[:, 2, :]
w[:, M, :] = 2*w[:, M-1, :] - w[:, M-2, :]
w[:, :, 0] = 2*w[:, :, 1] - w[:, :, 2]
w[:, :, M] = 2*w[:, :, M-1] - w[:, :, M-2]


a_1 = 1 + k * (r/2 + ((r-sk_q) * xh) + ((sk_sigma*xh)**2))
b_1 = (-1) * k*((sk_sigma*xh)**2)/(2)
c_1 = -k * (((r-sk_q) * xh) + (sk_sigma*xh)**2/(2))

a_2 = 1 + k * (r/2 + ((r-na_q) * xh)/h + ((na_sigma*xh)**2))
b_2 = (-1) * k*((na_sigma*xh)**2)/(2)
c_2 = -k * (((r-na_q) * xh)/h + (na_sigma*xh)**2/(2))


d_1 = np.zeros((N, M-1, M-1))
d_2 = np.zeros((N, M-1, M-1))

for t_idx in range(0, N):
    
    for y_idx in range(1, M-1):
    
        d_1[t_idx,:,y_idx-1] = u[t_idx, :, y_idx][1:-1] + ((k*rho*sk_sigma*na_sigma*xh[1:M]*xh[1:M])/(2*4)) * (u[t_idx,:,y_idx+1][2:] + u[t_idx,:,y_idx-1][0:-2] - u[t_idx,:,y_idx+1][0:-2] - u[t_idx,:,y_idx-1][2:])
        u[t_idx,:,y_idx][1:-1] = Thomas_Algorithm(a_1, b_1, c_1 ,d_1[t_idx,:,y_idx-1])

        d_2[t_idx,:,y_idx-1] = u[t_idx, :, y_idx][1:-1] + ((k*rho*sk_sigma*na_sigma*xh[1:M]*xh[1:M])/(2*4)) * (u[t_idx,:,y_idx+1][2:] + u[t_idx,:,y_idx-1][0:-2] - u[t_idx,:,y_idx+1][0:-2] - u[t_idx,:,y_idx-1][2:])
        u[t_idx,:,y_idx+1][1:-1] = Thomas_Algorithm(a_2, b_2, c_2, d_2[t_idx,:,y_idx-1])

u[0,:]


array([[ 0.00000000e+00,  1.00000000e-02,  2.00000000e-02, ...,
         1.30600000e+00,  1.30600000e+00,  1.30600000e+00],
       [ 0.00000000e+00,  9.99577179e-03,  9.98732202e-03, ...,
         5.22667521e-02,  5.26370013e-02,  1.30600000e+00],
       [ 0.00000000e+00,  9.99577179e-03,  9.98732073e-03, ...,
         2.11828241e-02,  2.14777200e-02,  1.30600000e+00],
       ...,
       [ 0.00000000e+00,  2.95298053e-03,  2.65678117e-03, ...,
        -5.96881995e-02, -5.50753135e-02,  1.30600000e+00],
       [ 0.00000000e+00,  1.16983607e-03,  5.94024540e-04, ...,
        -8.62151371e-02, -8.85388877e-02,  1.30600000e+00],
       [ 0.00000000e+00,  1.00000000e-02,  2.00000000e-02, ...,
         1.30600000e+00,  1.30600000e+00,  1.30600000e+00]])

In [423]:
u[10,10,]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:

u[0,:,1][1:-1] = Thomas_Algorithm(a_1, b_1, c_1 ,d_1[0,:,0])

d_1[0,:,0] = u[0, :, 1][1:-1] + ((k*rho*sk_sigma*na_sigma*xh[1:M]*xh[1:M])/(2*4)) * (u[0,:,2][2:] + u[0,:,0][0:-2] - u[0,:,2][0:-2] - u[0,:,0][2:])
#d_2

In [ ]:
'''Universal Boundary Condition'''
x = np.arange(M+1) / M #가격 분할 노드

sk_u = np.zeros((N+1, M+1)) #SK하이닉스 Mesh 생성
na_u = np.zeros((N+1, M+1)) #NAVER Mesh 생성 
sk_w = np.zeros((N+1, M+1)) #SK하이닉스 Mesh 생성
na_w = np.zeros((N+1, M+1)) #NAVER Mesh 생성 

sk_u[0,:] = np.where(x>B, 1 + ctag[-1], x).reshape(1, M+1)
na_u[0,:] = np.where(x>B, 1 + ctag[-1], x).reshape(1, M+1)

sk_w[0,:] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(1, M+1) #
na_w[0,:] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(1, M+1)



In [364]:
u[0, :, 4][1:-1].shape, u[0, :, 1][0:-2].shape, u[0, :, 1][2:].shape, 

((99,), (99,), (99,))

array([9.99577179e-03, 9.99157160e-03, 9.98231325e-03, 9.96903877e-03,
       9.95223060e-03, 9.93219063e-03, 9.90913359e-03, 9.88322365e-03,
       9.85459230e-03, 9.82334839e-03, 9.78958417e-03, 9.75337936e-03,
       9.71480385e-03, 9.67391967e-03, 9.63078243e-03, 9.58544240e-03,
       9.53794536e-03, 9.48833328e-03, 9.43664480e-03, 9.38291573e-03,
       9.32717937e-03, 9.26946678e-03, 9.20980708e-03, 9.14822763e-03,
       9.08475420e-03, 9.01941114e-03, 8.95222152e-03, 8.88320723e-03,
       8.81238908e-03, 8.73978692e-03, 8.66541968e-03, 8.58930544e-03,
       8.51146155e-03, 8.43190460e-03, 8.35065056e-03, 8.26771475e-03,
       8.18311193e-03, 8.09685631e-03, 8.00896159e-03, 7.91944100e-03,
       7.82830735e-03, 7.73557298e-03, 7.64124987e-03, 7.54534960e-03,
       7.44788343e-03, 7.34886224e-03, 7.24829663e-03, 7.14619688e-03,
       7.04257298e-03, 6.93743465e-03, 6.83079135e-03, 6.72265229e-03,
       6.61302645e-03, 6.50192258e-03, 6.38934920e-03, 6.27531464e-03,
      

True

In [318]:
# a = 1 + k * (r/2 + (sk_mu * xh)/h + ((sk_sigma*xh)**2)/(h**2))
# b = (-1) * k*((sk_sigma*xh)**2)/(4*h**2)
# c = -k * ((sk_mu * xh)/2 + (sk_sigma*xh)**2/((2*h)**2))

# a.shape, b.shape, c.shape

# a[1] = a[1] + 2*b[1]
# c[1] = c[1] - b[1]
# a[M-1] = a[M-1] + 2*c[M-1]
# b[M-1] = b[M-1] - c[M-1]

# d = np.zeros((M-1, M-1, M-1))
# a = a[1:M]
# b = b[1:M]
# c = c[1:M]

In [332]:
xh.shape

(101,)

In [334]:
u[0, :, 1][1:M].shape

(99,)

In [335]:
u[0, :, M-1][1:M] + k * na_mu * xh

ValueError: operands could not be broadcast together with shapes (99,) (101,) 

In [291]:
'''99, 1시작, 99엔드, 0과 100이 기븐'''
d = np.zeros((N-1, M-1, M-1))

for t_idx in range(0, N-1): #1~100이니까 99임
    
    for y_idx in range (0, M-1): #1~100이니까 99임 (양쪽에서 계산 들어갈것)
        
        d[t_idx, :, y_idx] = u[t_idx, :, y_idx] + k * na_mu * 
        
        
d


array([[[0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ],
        [0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ],
        [0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ],
        ...,
        [0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ],
        [0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ],
        [0.00999958, 0.01999958, 0.02999958, ..., 1.306     ,
         1.306     , 1.306     ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0. 

In [ ]:
d
Thomas_Algorithm(a, b, c, d)

In [184]:
dt_n_arr = dt_arr + dt/2
dt_n_arr

array([0.015, 0.045, 0.075, 0.105, 0.135, 0.165, 0.195, 0.225, 0.255,
       0.285, 0.315, 0.345, 0.375, 0.405, 0.435, 0.465, 0.495, 0.525,
       0.555, 0.585, 0.615, 0.645, 0.675, 0.705, 0.735, 0.765, 0.795,
       0.825, 0.855, 0.885, 0.915, 0.945, 0.975, 1.005, 1.035, 1.065,
       1.095, 1.125, 1.155, 1.185, 1.215, 1.245, 1.275, 1.305, 1.335,
       1.365, 1.395, 1.425, 1.455, 1.485, 1.515, 1.545, 1.575, 1.605,
       1.635, 1.665, 1.695, 1.725, 1.755, 1.785, 1.815, 1.845, 1.875,
       1.905, 1.935, 1.965, 1.995, 2.025, 2.055, 2.085, 2.115, 2.145,
       2.175, 2.205, 2.235, 2.265, 2.295, 2.325, 2.355, 2.385, 2.415,
       2.445, 2.475, 2.505, 2.535, 2.565, 2.595, 2.625, 2.655, 2.685,
       2.715, 2.745, 2.775, 2.805, 2.835, 2.865, 2.895, 2.925, 2.955,
       2.985, 3.015])

In [161]:
a = 1 + k

-0.06944230147199999

In [855]:
r = 0.0282 #발행일 8월 31일의 90일 CD금리

sk_q = 0.0162 #8월 31일의 sk하이닉스 배당수익률
na_q = 0.0022 #8월 31일의 네이버 배당수익률

rho = 0.5287 #투자 설명서의 상관계수
sk_sigma = 0.3888 #투자설명서의 SK하이닉스 변동성
na_sigma = 0.3962 #투자설명서의 NAVER 변동성

sk_0 = 1 #계산의 편의를 위해, 현재시점의 SK하이닉스의 가격을 1로 잡음
na_0 = 1 #계산의 편의를 위해, 현재시점의 NAVER의 가격을 1로 잡음

sk_mu = r - sk_q - rho * sk_sigma * na_sigma
na_mu = r - na_q - rho * na_sigma * sk_sigma

'''Given tags'''
B = 0.5
ctag = np.array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])
ktag = np.array([0.85, 0.85, 0.80, 0.80, 0.75, 0.75])
ttag = np.array([0.5, 1, 1.5, 2, 2.5, 3])
Rtag = 2 * N/len(ktag) * ttag


'''For Mesh'''

T = 3
sk_0 = 1
na_0 = 1
sk_max = sk_0 * 3
na_max = na_0 * 3

dx = 0.01
M = int(np.round((sk_max-sk_0)/dx)) #기초자산 주가 간격
N = 360 * T #만기시 간격

#dx = sk_max / M
dt = 1 #하루니까

k = dt
h = dx
a = k/h**2

xtag = np.where(xh==sk_0)

xh = np.arange(M+1)
x = np.full((M+1, M+1), xh) * dx


u = np.zeros((N+1, M+1, M+1)) #시간, SK하이닉스, NAVER의 노드
w = np.zeros((N+1, M+1, M+1))


u[0, :] = np.where(x>B, 1 + ctag[-1], x).reshape(M+1, M+1)
w[0, :] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(M+1, M+1)

u[:, 0, :] = 2*u[:, 1, :] - u[:, 2, :]
u[:, M, :] = 2*u[:, M-1, :] - u[:, M-2, :]
u[:, :, 0] = 2*u[:, :, 1] - u[:, :, 2]
u[:, :, M] = 2*u[:, :, M-1] - u[:, :, M-2]

w[:, 0, :] = 2*w[:, 1, :] - w[:, 2, :]
w[:, M, :] = 2*w[:, M-1, :] - w[:, M-2, :]
w[:, :, 0] = 2*w[:, :, 1] - w[:, :, 2]
w[:, :, M] = 2*w[:, :, M-1] - w[:, :, M-2]



In [872]:
# a_i = 1 + k*((r/2 + (r-sk_q) * xh / h) + (sk_sigma**2 * xh**2) / (h**2))
# b_i = -1 * k * (sk_sigma*xh)**2/(2*(h**2))
# c_i = -1 * k * ((r-sk_q) * xh / h + (sk_sigma**2 * xh**2) / (2 * (h**2)))

# a_j = 1 + k*((r/2 + (r-na_q) * xh / h) + (na_sigma**2 * xh**2) / (h**2))
# b_j = -1 * k * (na_sigma*xh)**2/(2*(h**2))
# c_j = -1 * k * ((r-na_q) * xh / h + (na_sigma**2 * xh**2) / (2 * (h**2)))


# corr_eff = k * rho * sk_sigma * na_sigma * xh * xh / (8 * h**2)

a_i = 1 + k*((r/2 + (r-sk_q) * xh ) + (sk_sigma**2 * xh**2) )
b_i = -1 * k * (sk_sigma*xh)**2/(2)
c_i = -1 * k * ((r-sk_q) * xh + (sk_sigma**2 * xh**2) / (2))

a_j = 1 + k*((r/2 + (r-na_q) * xh) + (na_sigma**2 * xh**2) )
b_j = -1 * k * (na_sigma*xh)**2/(2)
c_j = -1 * k * ((r-na_q) * xh + (na_sigma**2 * xh**2) / (2))


corr_eff = k * rho * sk_sigma * na_sigma * xh * xh / (8) 

corr_eff = corr_eff[1:M]

a_i = a_i[1:M]
b_i = b_i[1:M]
c_i = c_i[1:M]
a_j = a_j[1:M]
b_j = b_j[1:M]
c_j = c_j[1:M]


In [873]:
corr_eff.shape

(199,)

In [874]:
n = 0 

u[n, 1:M, y_idx] + corr_eff * (u[n, 2:, y_idx+1]+u[n, 0:M-1, y_idx-1]-u[n, 0:M-1, y_idx+1]-u[n, 2:, y_idx-1])

array([ 4.19131819e+182,  9.01367691e+182,  1.44655571e+183,
        2.05449025e+183,  2.72494691e+183,  3.45768887e+183,
        4.25246840e+183,  5.10902719e+183,  6.02709592e+183,
        7.00639371e+183,  8.04662728e+183,  9.14749008e+183,
        1.03086612e+184,  1.15298045e+184,  1.28105669e+184,
        1.41505777e+184,  1.55494469e+184,  1.70067641e+184,
        1.85220967e+184,  2.00949887e+184,  2.17249593e+184,
        2.34115006e+184,  2.51540772e+184,  2.69521233e+184,
        2.88050419e+184,  3.07122029e+184,  3.26729413e+184,
        3.46865554e+184,  3.67523055e+184,  3.88694117e+184,
        4.10370526e+184,  4.32543632e+184,  4.55204338e+184,
        4.78343076e+184,  5.01949798e+184,  5.26013955e+184,
        5.50524482e+184,  5.75469786e+184,  6.00837728e+184,
        6.26615611e+184,  6.52790164e+184,  6.79347531e+184,
        7.06273260e+184,  7.33552289e+184,  7.61168935e+184,
        7.89106887e+184,  8.17349195e+184,  8.45878264e+184,
        8.74675843e+184,

In [870]:
u_star = u
w_star = w

for n in range(0, N):

    '''for early redemption'''
    for j in range(len(ktag)-1):
        if t_idx == Rtag[j]:
            dummy = int(x0tag * ctag[len(ctag)-2-j])
            
            u[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]
            w[t_idx, dummy:, dummy:] = 1 + ctag[len(ctag)-2-j]

    for y_idx in range(1, M):

        d_x_i_u = u[n, 1:M, y_idx] + corr_eff * (u[n, 2:, y_idx+1]+u[n, 0:M-1, y_idx-1]-u[n, 0:M-1, y_idx+1]-u[n, 2:, y_idx-1])
        u_star[n, 1:M, y_idx] = Thomas_Algorithm(a_i, b_i, c_i, d_x_i_u)

        d_x_i_w = w[n, 1:M, y_idx] + corr_eff * (w[n, 2:, y_idx+1]+w[n, 0:M-1, y_idx-1]-w[n, 0:M-1, y_idx+1]-w[n, 2:, y_idx-1])
        w_star[n, 1:M, y_idx] = Thomas_Algorithm(a_i, b_i, c_i, d_x_i_w)

        
    for x_idx in range(1, M):

        d_x_j_u = u_star[n, 1:M, y_idx] + corr_eff * (u_star[n, 2:, y_idx+1]+u_star[n, 0:M-1, y_idx-1]-u_star[n, 0:M-1, y_idx+1]-u_star[n, 2:, y_idx-1])
        u[n+1, x_idx, 1:M] = Thomas_Algorithm(a_j, b_j, c_j, d_x_j_u)
        d_x_j_w = w_star[n, 1:M, y_idx] + corr_eff * (w_star[n, 2:, y_idx+1]+w_star[n, 0:M-1, y_idx-1]-w_star[n, 0:M-1, y_idx+1]-w_star[n, 2:, y_idx-1])
        w[n+1, x_idx, 1:M] = Thomas_Algorithm(a_j, b_j, c_j, d_x_j_w)

    u[n+1, 1:Btag, 1:Btag] = w[n+1, 1:Btag, 1:Btag]

    u[n+1, 0, :] = 2*u[n+1, 1, :] - u[n+1, 2, :]
    u[n+1, M, :] = 2*u[n+1, M-1, :] - u[n+1, M-2, :]
    u[n+1, :, 0] = 2*u[n+1, :, 1] - u[n+1, :, 2]
    u[n+1, :, M] = 2*u[n+1, :, M-1] - u[n+1, :, M-2]

    w[n+1, 0, :] = 2*w[n+1, 1, :] - w[n+1, 2, :]
    w[n+1, M, :] = 2*w[n+1, M-1, :] - w[n+1, M-2, :]
    w[n+1, :, 0] = 2*w[n+1, :, 1] - w[n+1, :, 2]
    w[n+1, :, M] = 2*w[n+1, :, M-1] - w[n+1, :, M-2]
        

/var/folders/0k/zyr795gx3hl6x1nytrrntcnc0000gn/T/ipykernel_4189/259304994.py:7: RuntimeWarning: overflow encountered in double_scalars
  dc[it] = d[it] - b[it]*(dc[it-1]/ac[it-1])
/var/folders/0k/zyr795gx3hl6x1nytrrntcnc0000gn/T/ipykernel_4189/139115319.py:16: RuntimeWarning: invalid value encountered in subtract
  d_x_i_u = u[n, 1:M, y_idx] + corr_eff * (u[n, 2:, y_idx+1]+u[n, 0:M-1, y_idx-1]-u[n, 0:M-1, y_idx+1]-u[n, 2:, y_idx-1])
/var/folders/0k/zyr795gx3hl6x1nytrrntcnc0000gn/T/ipykernel_4189/139115319.py:19: RuntimeWarning: invalid value encountered in subtract
  d_x_i_w = w[n, 1:M, y_idx] + corr_eff * (w[n, 2:, y_idx+1]+w[n, 0:M-1, y_idx-1]-w[n, 0:M-1, y_idx+1]-w[n, 2:, y_idx-1])


KeyboardInterrupt: 

In [683]:

u = np.zeros((N, M+1, M+1)) #시간, SK하이닉스, NAVER의 노드
w = np.zeros((N, M+1, M+1))

u[0, :] = np.where(x>B, 1 + ctag[-1], x).reshape(M+1, M+1)
w[0, :] = np.where(x>=ktag[-1], 1 + ctag[-1], x).reshape(M+1, M+1)

u[:, 0, :] = 2*u[:, 1, :] - u[:, 2, :]
u[:, M, :] = 2*u[:, M-1, :] - u[:, M-2, :]
u[:, :, 0] = 2*u[:, :, 1] - u[:, :, 2]
u[:, :, M] = 2*u[:, :, M-1] - u[:, :, M-2]

w[:, 0, :] = 2*w[:, 1, :] - w[:, 2, :]
w[:, M, :] = 2*w[:, M-1, :] - w[:, M-2, :]
w[:, :, 0] = 2*w[:, :, 1] - w[:, :, 2]
w[:, :, M] = 2*w[:, :, M-1] - w[:, :, M-2]


In [910]:
import numpy as np

class OSM_2Stock_ELS:

    def __init__(self, inputs):

        self.FV   = inputs['facevalue']                # Face Value

        self.x0   = inputs['initial_price1']           # Initial Prices for Underlyings
        self.y0   = inputs['initial_price2']           
        
        max_Sx    = self.x0 * 3                        # Set Max Value for Underlyings as 300%
        max_Sy    = self.y0 * 3                        

        self.xVol = inputs['vol1']                     # Volatilities for Underlyings
        self.yVol = inputs['vol2']  

        self.rho  = inputs['rho']                      # Correlation between x and y
        self.r    = inputs['r']                        # Risk-Free Rate 
        self.x_q  = inputs['x_q']
        self.y_q  = inputs['y_q']

        self.intv = inputs['intervals'] + 1            # Number of Intervals for Underlying

        self.hx   = max_Sx / self.intv                 # Intervals for Underlyings
        self.hy   = max_Sy / self.intv                

        self.x    = np.linspace(0, max_Sx, self.intv)  # The First Underlying
        self.y    = np.linspace(0, max_Sy, self.intv)  # The Second Underlying

        T         = inputs['maturity']                 # Maturity
        self.numT = 360 * T                            # Number of Time Steps
        self.k    = T / self.numT                      # Interval for Time Stpes 


        # Coupons on Early Exercise Date
        coupons = inputs['coupon']; coupons.reverse()
        self.coupon_rate = np.array(coupons)

        # Strike Prices
        strikes = inputs['strike']; strikes.reverse()
        self.strike_price=np.array(strikes)

        # Days to Early Termination
        self.step = np.array([np.rint(self.numT/6),   np.rint(2*self.numT/6), np.rint(3*self.numT/6), \
                              np.rint(4*self.numT/6), np.rint(5*self.numT/6), self.numT+1])
        
        # Knock-In Barrier
        self.barrier=inputs['barrier']

        
    def thomas(self, a, b, c, d):

        n = len(d)
        vec=np.zeros(n)
        [aa,bb,cc,dd] = map(np.array, [a, b, c, d])
        
        for i in range(1,n):
            mult   = aa[i]/bb[i-1]
            bb[i] -= mult*cc[i-1]
            dd[i] -= mult*dd[i-1]
        
        vec[n-1] = dd[n-1]/bb[n-1]

        for i in range(n-2, -1, -1):
            vec[i] = (dd[i]-cc[i]*vec[i+1])/bb[i]

        return vec

    def set_initial_values(self):
        # Set Initial Values considering the strike price at the maturity

        dim = [self.intv, self.intv]
        [u, v] = map(np.zeros, [dim, dim])

        for i in range(self.intv):
            for j in range(self.intv):

                if (self.x[i] < self.barrier * self.x0) or (self.y[j] < self.barrier*self.y0):

                    if np.minimum(self.x[i], self.y[j]) == self.x[i]:
                        u[i,j] = np.minimum(self.x[i], self.y[j]) / self.x0 * self.FV
                        v[i,j] = np.minimum(self.x[i], self.y[j]) / self.x0 * self.FV

                    elif self.y[j] < self.barrier*self.y0:
                        u[i,j] = np.minimum(self.x[i], self.y[j]) / self.y0 * self.FV
                        v[i,j] = np.minimum(self.x[i], self.y[j]) / self.y0 * self.FV



                elif (self.x[i] <= self.strike_price[0] * self.x0) or (self.y[j] <= self.strike_price[0] * self.y0):
                    
                    if self.x[i] <= self.strike_price[0] * self.x0:
                        u[i,j] = self.FV * (1 + self.coupon_rate[0]) 
                        v[i,j] = np.minimum(self.x[i], self.y[j]) / self.x0 * self.FV

                    elif self.y[j] <= self.strike_price[0] * self.y0:
                        u[i,j] = self.FV * (1 + self.coupon_rate[0]) 
                        v[i,j] = np.minimum(self.x[i], self.y[j]) / self.y0 * self.FV

                else:
                    u[i,j] = self.FV * (1 + self.coupon_rate[0])
                    v[i,j] = self.FV * (1 + self.coupon_rate[0])

        return u,v


    def coefficients(self):
        # Coefficients for Operator Splitting Method

        [ax, bx, cx, ay, by, cy] = map(np.zeros, [self.intv-2, self.intv-2, self.intv-2, \
                                                  self.intv-2, self.intv-2, self.intv-2])

        ax[:] = -0.5 * (self.xVol * self.x[1:self.intv-1]/self.hx) ** 2 + (self.r - self.x_q)*self.x[1:self.intv-1] / (2*self.hx)
        bx[:] = 1/self.k  + (self.xVol * self.x[1:self.intv-1]/self.hx) ** 2 + (self.r - self.x_q)*0.5
        cx[:] = -0.5 * (self.xVol * self.x[1:self.intv-1]/self.hx) ** 2 - (self.r - self.x_q)*self.x[1:self.intv-1] / (2*self.hx)

        ay[:] = -0.5 * (self.yVol * self.y[1:self.intv-1]/self.hy)**2 + (self.r - self.x_q)*self.y[1:self.intv-1] / (2*self.hy)
        by[:] = 1/self.k  + (self.yVol * self.y[1:self.intv-1]/self.hy)**2 + (self.r - self.y_q) / 2
        cy[:] = -0.5 * (self.yVol * self.y[1:self.intv-1]/self.hy)**2 - (self.r - self.x_q)*self.y[1:self.intv-1] / (2*self.hy)

        return ax,bx,cx,ay,by,cy


    def os_solver(self, uv):

        os_coefficients = self.coefficients()
        ax = os_coefficients[0]; bx = os_coefficients[1]; cx = os_coefficients[2]
        ay = os_coefficients[3]; by = os_coefficients[4]; cy = os_coefficients[5]

        dx = np.zeros(self.intv-2); dy = np.zeros(self.intv-2)

        rep_uv = uv
        # Solve with x-direction
        for j in range(1, self.intv-1):
            dx[0:self.intv-1] = rep_uv[1:self.intv-1, j] / self.k                           + \
                                (self.rho * self.xVol  * self.yVol                          * \
                                 self.x[1:self.intv-1] * self.y[j]) / 2                     * \
                                                                                              \
                                (rep_uv[2:self.intv, j+1] - rep_uv[2:self.intv, j-1]        - \
                                 rep_uv[0:self.intv-2, j+1] + rep_uv[0:self.intv-2, j-1] )  / \
                                 (4*self.hx ** 2)

            uv[1:self.intv-1, j] = self.thomas(ax, bx, cx, dx)
        

        rep_uv = uv
        # Solve with y-direction
        for i in range(1, self.intv-1):
            dy[0:self.intv-1] = rep_uv[i, 1:self.intv-1] / self.k                       + \
                                (self.rho  * self.xVol * self.yVol                      * \
                                 self.x[i] * self.y[1:self.intv-1]) / 2                 * \
                                                                                          \
                                (rep_uv[i+1, 2:self.intv] - rep_uv[i+1,0:self.intv-2]   - \
                                 rep_uv[i-1,2:self.intv] + rep_uv[i-1,0:self.intv-2])   / \
                                 (4*self.hy**2)
                            
            uv[i, 1:self.intv-1] = self.thomas(ay, by, cy, dy)

        return uv


    def ELS_Price(self, bumper):

        init_uv = self.set_initial_values()
        u  = init_uv[0]; v = init_uv[1]

        counter = 0
        for n in range(self.numT):
            
            if (n==self.step[counter]):
                # Find indices of minimum x,y satisfy the early termination condition
                xBarrier_i = np.min(np.where(self.x >= self.x0 * self.strike_price[counter+1]))
                yBarrier_j = np.min(np.where(self.y >= self.y0 * self.strike_price[counter+1]))
                
                # Update values with applying coupons on the early termination dates
                u[xBarrier_i:self.intv-1, yBarrier_j:self.intv-1] = self.FV * (1 + self.coupon_rate[counter+1])
                v[xBarrier_i:self.intv-1, yBarrier_j:self.intv-1] = self.FV * (1 + self.coupon_rate[counter+1])

                counter += 1

            # Indices of x,y under the knock-in barrier
            xBarrier_i = np.min(np.where(self.x >= self.x0 * self.barrier))
            yBarrier_j = np.min(np.where(self.y >= self.y0 * self.barrier))

            # Update U values under the knock-in barrier with corresponding V vlaues
            u[:, 0:yBarrier_j+1] = v[:,0:yBarrier_j+1]
            u[0:xBarrier_i+1, :] = v[0:xBarrier_i+1, :]
            
            # Solve with OS FDM Algorithm
            u = self.os_solver(u)
            v = self.os_solver(v)

        
        x_i = np.where((self.x0-1 <= self.x) & (self.x < self.x0+1))
        y_i = np.where((self.y0-1 <= self.y) & (self.y < self.y0+1))

        return u[x_i + np.array(bumper), y_i + np.array(bumper)][0][0]



class Greeks:

    def __init__(self, inputs):
        self.inputs = inputs

    def delta(self):        
        V = OSM_2Stock_ELS(self.inputs)

        v1 = V.ELS_Price(1)
        v2 = V.ELS_Price(-1)

        # h 가중평균
        h = (V.hx * 0.1451) + (V.hy * (1 - 0.1451))

        return (v1 - v2) / (2*h)

    def gamma(self):
        V = OSM_2Stock_ELS(self.inputs)

        v1 = V.ELS_Price(1)
        v2 = V.ELS_Price(0)
        v3 = V.ELS_Price(-1)

        # h 가중평균
        h = (V.hx * 0.1451) + (V.hy * (1 - 0.1451))

        return (v1 - 2*v2 + v3) / (h**2)



# NH투자증권 제19904회 2-Stock Step Down ELS
NH_ELS_19904 = {
    'facevalue'     : 10000,      # Face Value
    'initial_price1': 100,       # Initial Price for Starbucks 88.38
    'initial_price2': 100,      # Initial Price for Netflix  483.86
    'intervals'     : 90,          # Number of Intervals x, y
    'vol1'          : 0.3888,      # Volatility for Starbucks
    'vol2'          : 0.3962,      # Volatility for Netflix
    'rho'           : 0.5287,       # Correlation between Underlyings
    'x_q'           : 0.013,      # 180영업일의 평균 배당수익률 (SK하이닉스)
    'y_q'           : 0.0016,      # 180영업일의 평균 배당수익률 (NAVER)
    'maturity'      : 3,           # Maturity in Years
    'r'             : 0.0282,   # Risk-Free Rates: 3M LIBOR as of Oct 22 2020
    'barrier'       : 0.5,        # Knock-In Barrier
    'coupon'        : [0.051, 0.102, 0.153, 0.204, 0.255, 0.306],  # Coupons
    'strike'        : [0.85, 0.85, 0.80, 0.80, 0.75, 0.75],  # Strike Prices
}

NH = OSM_2Stock_ELS(NH_ELS_19904)
print("The ELS Price is ", NH.ELS_Price(0))

NH_Greeks = Greeks(NH_ELS_19904)
print("Delta is ", NH_Greeks.delta())
print("Gamma is ,", NH_Greeks.gamma())

The ELS Price is  8897.683917502212
Delta is  86.64099929601448
Gamma is , -1.985277714180559


In [984]:
x_vol         = 0.3888      # Volatility for Starbucks
y_vol         = 0.3962      # Volatility for Netflix
rho           = 0.5287       # Correlation between Underlyings
x_q           = 0.013      # 180영업일의 평균 배당수익률 (SK하이닉스)
y_q           = 0.0016      # 180영업일의 평균 배당수익률 (NAVER)
T             = 3           # Maturity in Years
rf            = 0.0282   # Risk-Free Rates= 3M LIBOR as of Oct 22 2020
barrier       = 0.5        # Knock-In Barrier
coupon        = np.array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])[::-1]  # Coupons
strike        = np.array([0.85, 0.85, 0.80, 0.80, 0.75, 0.75])[::-1]  # Strike Prices

facevalue     = 10000
x0_price      = 100
y0_price      = 100
x0_max        = x0_price * 3                        
y0_max        = y0_price * 3  
parsing       = 91         # Number of Intervals x y
hx            = x0_max / parsing                 # Intervals for Underlyings
hy            = y0_max / parsing                

x             = np.linspace(0, x0_max, parsing)  # The First Underlying
y             = np.linspace(0, y0_max, parsing) 

numT          = 360 * T                            # Number of Time Steps
k             = T / numT
step = np.int0((np.linspace(0, T, 2 * T +1) * numT/3))[1:]



In [980]:
def Thomas_Algorithm(a, b, c, d):

    n = len(d)
    vec=np.zeros(n)
    [aa,bb,cc,dd] = map(np.array, [a, b, c, d])

    for i in range(1,n):
        mult   = aa[i]/bb[i-1]
        bb[i] -= mult*cc[i-1]
        dd[i] -= mult*dd[i-1]

    vec[n-1] = dd[n-1]/bb[n-1]

    for i in range(n-2, -1, -1):
        vec[i] = (dd[i]-cc[i]*vec[i+1])/bb[i]

    return vec

In [983]:
def os_solver(coef_list, u, parsing):

    coef = coef_list
    a_i = coef[0]
    b_i = coef[1]
    c_i = coef[2]
    a_j = coef[3]
    b_j = coef[4]
    c_j = coef[5]

    d_i = np.zeros(parsing-2)
    d_j = np.zeros(parsing-2)

    tmp = u
    # Solve with x-direction
    for j in range(1, parsing-1):
        d_i[0:parsing-1] = tmp[1:parsing-1, j] / k + (rho * x_vol * y_vol *  x[1:parsing-1] * y[j]) / 2 * \
                          (tmp[2:parsing, j+1] - tmp[2:parsing, j-1] - tmp[0:parsing-2, j+1] + tmp[0:parsing-2, j-1] ) / \
                          (4*hx ** 2)
        u[1:parsing-1, j] = Thomas_Algorithm(a_i, b_i, c_i, d_i)
    

    tmp = u
    # Solve with y-direction
    for i in range(1, parsing-1):
        d_j[0:parsing-1] = tmp[i, 1:parsing-1] / k + (rho * x_vol * y_vol * x[i] * y[1:parsing-1]) / 2 * \
                          (tmp[i+1, 2:parsing] - tmp[i+1,0:parsing-2] - tmp[i-1,2:parsing] + tmp[i-1,0:parsing-2])  / \
                          (4*hy**2)
                        
        u[i, 1:parsing-1] = Thomas_Algorithm(a_j, b_j, c_j, d_j)

    return u

In [982]:
def os_solver(coef_list, uv, parsing):

    os_coefficients = coef_list
    ax = os_coefficients[0]; bx = os_coefficients[1]; cx = os_coefficients[2]
    ay = os_coefficients[3]; by = os_coefficients[4]; cy = os_coefficients[5]

    dx = np.zeros(parsing-2); dy = np.zeros(parsing-2)

    rep_uv = uv
    # Solve with x-direction
    for j in range(1, parsing-1):
        dx[0:parsing-1] = rep_uv[1:parsing-1, j] / k                           + \
                            (rho * x_vol  * y_vol                          * \
                                x[1:parsing-1] * y[j]) / 2                     * \
                                                                                            \
                            (rep_uv[2:parsing, j+1] - rep_uv[2:parsing, j-1]        - \
                                rep_uv[0:parsing-2, j+1] + rep_uv[0:parsing-2, j-1] )  / \
                                (4*hx ** 2)

        uv[1:parsing-1, j] = Thomas_Algorithm(ax, bx, cx, dx)
    

    rep_uv = uv
    # Solve with y-direction
    for i in range(1, parsing-1):
        dy[0:parsing-1] = rep_uv[i, 1:parsing-1] / k                       + \
                            (rho  * x_vol * y_vol                      * \
                                x[i] * y[1:parsing-1]) / 2                 * \
                                                                                        \
                            (rep_uv[i+1, 2:parsing] - rep_uv[i+1,0:parsing-2]   - \
                                rep_uv[i-1,2:parsing] + rep_uv[i-1,0:parsing-2])   / \
                                (4*hy**2)
                        
        uv[i, 1:parsing-1] = Thomas_Algorithm(ay, by, cy, dy)

    return uv

In [985]:
dim = [parsing, parsing]
[u, w] = map(np.zeros, [dim, dim])

for i in range(parsing):
    for j in range(parsing):

        if (x[i] < barrier * x0_price) or (y[j] < barrier*y0_price):

            if np.minimum(x[i], y[j]) == x[i]:
                u[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue
                w[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue

            elif y[j] < barrier*y0_price:
                u[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue
                w[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue



        elif (x[i] <= strike[0] * x0_price) or (y[j] <= strike[0] * y0_price):
            
            if x[i] <= strike[0] * x0_price:
                u[i,j] = facevalue * (1 + coupon[0]) 
                w[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue

            elif y[j] <= strike[0] * y0_price:
                u[i,j] = facevalue * (1 + coupon[0]) 
                w[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue

        else:
            u[i,j] = facevalue * (1 + coupon[0])
            w[i,j] = facevalue * (1 + coupon[0])

In [986]:
[a_i, b_i, c_i, a_j, b_j, c_j] = map(np.zeros, [parsing-2, parsing-2, parsing-2, \
                                            parsing-2, parsing-2, parsing-2])

a_i[:] = -0.5 * (x_vol * x[1:parsing-1]/hx) ** 2 + (r - x_q)*x[1:parsing-1] / (2*hx)
b_i[:] = 1/k  + (x_vol * x[1:parsing-1]/hx) ** 2 + (r - x_q)*0.5
c_i[:] = -0.5 * (x_vol * x[1:parsing-1]/hx) ** 2 - (r - x_q)*x[1:parsing-1] / (2*hx)

a_j[:] = -0.5 * (y_vol * y[1:parsing-1]/hy)**2 + (r - x_q)*y[1:parsing-1] / (2*hy)
b_j[:] = 1/k  + (y_vol * y[1:parsing-1]/hy)**2 + (r - y_q) / 2
c_j[:] = -0.5 * (y_vol * y[1:parsing-1]/hy)**2 - (r - x_q)*y[1:parsing-1] / (2*hy)

coef_list = [a_i, b_i, c_i, a_j, b_j, c_j]

In [987]:
counter = 0
for n in range(numT):
    
    if (n==step[counter]):
        # Find indices of minimum x,y satisfy the early termination condition
        xBarrier_i = np.min(np.where(x >= x0_price * strike[counter+1]))
        yBarrier_j = np.min(np.where(y >= y0_price * strike[counter+1]))
        
        # Update values with applying coupons on the early termination dates
        u[xBarrier_i:parsing-1, yBarrier_j:parsing-1] = facevalue * (1 + coupon[counter+1])
        w[xBarrier_i:parsing-1, yBarrier_j:parsing-1] = facevalue * (1 + coupon[counter+1])

        counter += 1

    # Indices of x,y under the knock-in barrier
    xBarrier_i = np.min(np.where(x >= x0_price * barrier))
    yBarrier_j = np.min(np.where(y >= y0_price * barrier))

    # Update U values under the knock-in barrier with corresponding V vlaues
    u[:, 0:yBarrier_j+1] = w[:,0:yBarrier_j+1]
    u[0:xBarrier_i+1, :] = w[0:xBarrier_i+1, :]
    
    # Solve with OS FDM Algorithm
    u = os_solver(coef_list, u, parsing)
    w = os_solver(coef_list, w, parsing)


x_i = np.where((x0_price-1 <= x) & (x < x0_price+1))
y_i = np.where((y0_price-1 <= y) & (y < y0_price+1))

u[x_i + np.array(0), y_i + np.array(0)][0][0]

8897.683917502212

In [ ]:
'''From Market Condition'''
r = 0.0282 #발행일 8월 31일의 90일 CD금리

sk_q = 0.0162 #8월 31일의 sk하이닉스 배당수익률
na_q = 0.0022 #8월 31일의 네이버 배당수익률

rho = 0.5287 #투자 설명서의 상관계수
sk_sigma = 0.3888 #투자설명서의 SK하이닉스 변동성
na_sigma = 0.3962 #투자설명서의 NAVER 변동성

sk_0 = 1 #계산의 편의를 위해, 현재시점의 SK하이닉스의 가격을 1로 잡음
na_0 = 1 #계산의 편의를 위해, 현재시점의 NAVER의 가격을 1로 잡음

sk_mu = r - sk_q - rho * sk_sigma * na_sigma
na_mu = r - na_q - rho * na_sigma * sk_sigma

'''Given tags'''
B = 0.5
ctag = np.array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])
ktag = np.array([0.85, 0.85, 0.80, 0.80, 0.75, 0.75])
ttag = np.array([0.5, 1, 1.5, 2, 2.5, 3])

'''For Mesh'''
T = 3 #만기 3년


In [ ]:
parameter = {
    'facevalue'     : 10000,      # Face Value
    'x0_price'      : 100,       # Sk하이닉스 8월 31일 가격
    'y0_price'      : 100,      # Naver 8월 31일 가격
    'parsing'       : 30,          # Number of Intervals x, y
    'x0_vol'        : 0.3888,      # Volatility for SK하이닉스
    'y0_vol'        : 0.3962,      # Volatility for 네이버
    'x0_div'       : 0.0162,
    'y0_div'       : 0.0022,
    'rho'           : 0.5287,       # Correlation between Underlyings
    'maturity'      : 3,           # Maturity in Years
    'rf'             : 0.0282,   # Risk-Free Rates: 3M LIBOR as of Oct 22 2020
    'barrier'       : 0.5,        # Knock-In Barrier
    'coupon'        : [0.051, 0.102, 0.153, 0.204, 0.255, 0.306],  # Coupons
    'strike'        : [0.85, 0.85, 0.80, 0.80, 0.75, 0.75],  # Strike Prices
}